<a href="https://colab.research.google.com/github/Khadeja29/cs-412-Machine-Learning/blob/main/KhadejaIqbal_26242_ML202102_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 412 Machine Learning 2020 

# Assignment 3

100 pts

## Goal 

The goal of this assignment 

*  Introduction to working with text data
*  Gain experience with the Scikit-Learn library
*  Gain experience with Naive Bayes and Logistic Regression

## Dataset

**20 Newsgroup Dataset** is a collection 18846 documents which are about 20 different topics.


## Task
Build naive bayes and logistic regression classifiers with the scikit-learn library function to **classify** the documents about their content topic.

## Submission

Follow the instructions at the end.

# 1) Initialize

First, make a copy of this notebook in your drive

# 2) Load Dataset

The 20 Newsgroup Dataset exist on Scikit-Learn library.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
train_batch = fetch_20newsgroups(subset='train')
test_batch = fetch_20newsgroups(subset='test')

In [ ]:
# target groups you will be dealing with
target_groups = train_batch.target_names
target_groups

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
# creating training and test sets
train_x =  train_batch["data"]
train_y =  train_batch["target"]
test_x  =  test_batch["data"]
test_y  =  test_batch["target"]

In [ ]:
print(train_x[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [ ]:
print(target_groups[train_y[0]])

rec.autos


# Preprocess

In [ ]:
import re

In [ ]:
%%capture
import nltk
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
# You will use this function to preprocess your data. If you would like to add another preprocessing step in the function, please add it and mention about it in your report.
def preprocess(text):
  text = re.sub("[\w\d._]+@[^\s]+|[^\s]+\.[^\s]+|[^\s]+-[^\s]+|\d+|[^\w\s]","",text.lower().strip())
  text = ' '.join([stemmer.stem(word) for word in re.findall("\w+",text) if word not in stop_words])
  return text

In [ ]:
# Apply <preprocess> function on the training and test set 
preprocessed_train_x = [preprocess(sample) for sample in train_x]
preprocessed_test_x  = [preprocess(sample) for sample in test_x]

# Models

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

## Tune Naive Bayes

In [ ]:
# Create a CountVectorizer for NB with:
#     min_df = 50
#     max_df = 3000
#     stop_words = stop_words

vectorizer = CountVectorizer(min_df = 50,max_df = 3000,stop_words = stop_words)


In [ ]:
# Vectorize your training and test set
train_x_nb = vectorizer.fit_transform(preprocessed_train_x)
test_x_nb = vectorizer.transform(preprocessed_test_x)


In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

#Initiate the NB model with required components.
model =MultinomialNB()

#Set the hyperparameter space that will be scanned:
#   alpha = (0.1,0.5,1.0,5.0)
mnb_pipeline = Pipeline([
                         
                         ("clf", model)
])
hyperparameters = dict(
    clf__alpha = (0.1,0.5,1.0,5.0),
)
#Let the GridSearchCV scan the hyperparameter and find the best hyperparameter set that will maximize the scoring option.
#   cv = 3
#   scoring = "accuracy"
mnb_grid_search = GridSearchCV(mnb_pipeline,hyperparameters,cv=3,scoring="accuracy",n_jobs=-1)
mnb_grid_search.fit(train_x_nb,train_y)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('clf',
                                        MultinomialNB(alpha=1.0,
                                                      class_prior=None,
                                                      fit_prior=True))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'clf__alpha': (0.1, 0.5, 1.0, 5.0)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
# show the best score
mnb_grid_search.best_params_

{'clf__alpha': 0.5}

In [ ]:
# show the best parameter
mnb_grid_search.best_score_

0.8212835269890522

### Evaluate The Best Model for NB

In [ ]:
#Create your NB model with the best parameter set.

model_nb_final= MultinomialNB(alpha=0.5)
#Fit your model on training set.
model_nb_final = model_nb_final.fit(train_x_nb,train_y)

In [ ]:
# Make predictions on test set
predictions =model_nb_final.predict(test_x_nb)

In [ ]:
# Show your accuracy on test set
accuracy_score(test_y,predictions)

0.7555762081784386

## Tune Logistic Regresion

In [ ]:
# Create a CountVectorizer for LR with:
#     min_df = 50
#     max_df = 3000
#     stop_words = stop_words

vectorizer2 = CountVectorizer( min_df = 50,max_df = 3000,stop_words = stop_words)

In [ ]:
# Vectorizer your training and test set
train_x_lr = vectorizer2.fit_transform(preprocessed_train_x)
test_x_lr = vectorizer2.transform(preprocessed_test_x)


In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#Initiate the LR model:
#   max_iter=2000
model_lr= LogisticRegression(max_iter=2000)

# Set the hyperparameter space that will be scanned:
#     C = (0.001,0.01,0.1,1)     
mnb_pipeline = Pipeline([
                         
                         ("clf", model_lr)
])
hyperparameters = dict(
    clf__C = (0.001,0.01,0.1,1),
)

#Let the GridSearchCV scan the hyperparameter and find the best hyperparameter set that will maximize the scoring option.
#   cv = 3
#   scoring = "accuracy"

mnb_grid_search = GridSearchCV(mnb_pipeline,hyperparameters,cv=3,scoring="accuracy",n_jobs=-1)
mnb_grid_search.fit(train_x_lr,train_y)


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('clf',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                                           max_iter=2000,
                                                           multi_class='auto',
                                                           n_jobs=None,
                                                           penalty='l2',
                                                           random_state=None,
                                                

In [ ]:
# show the best score
mnb_grid_search.best_score_

0.8338344038554356

In [ ]:
# show the best parameter
mnb_grid_search.best_params_

{'clf__C': 0.1}

### Evaluate The Best Model for Logistic Regression

In [ ]:
#Create your LR model with the best parameter set.
model_lr_final =LogisticRegression(max_iter=2000, C=0.1)
#Fit your model on training set.
model_lr_final =model_lr_final.fit(train_x_lr,train_y)

In [ ]:
# Make predictions on test set
pred= model_lr_final.predict(test_x_lr)

In [ ]:
# Show your accuracy on test set
accuracy_score(test_y, pred)

# Feature Importances

In [ ]:
lr_df = pd.DataFrame(columns=target_groups)
nb_df = pd.DataFrame(columns=target_groups)

In [ ]:
# Find the each category's most important top 3 features (words) for LR model and show with a dataframe
def feature_impo(model,target_groups,vectorizer):
    feature= np.asarray(vectorizer.get_feature_names())
    for i, t in enumerate(target_groups):
      lr_df[t]= feature[np.argsort(model.coef_[i])[-3:]][::-1] 

feature_impo(model_lr_final,target_groups,vectorizer2)

In [ ]:
# It should look like this:
lr_df

In [ ]:
# Find the each category's most important top 3 features (words) for NB model and show with a dataframe
def feature_impo(model,target_groups,vectorizer):
    feature= np.asarray(vectorizer.get_feature_names())
    for i, t in enumerate(target_groups):
      nb_df[t]= feature[np.argsort(model.feature_log_prob_[i])[-3:]][::-1] 

feature_impo(model_nb_final,target_groups,vectorizer)

In [ ]:
# It should look like this:
nb_df

# **Notebook & Report**

Notebook: We may just look at your notebook results; so make sure each cell is run and outputs are there.

Report: Write an at most 1/2 page summary of your approach to this problem at the end of your notebook; this should be like an abstract of a paper or the executive summary.

Must include statements such as:

( Include the problem definition: 1-2 lines )

(Talk about any preprocessing you did, explain your reasoning)

(Talk about train/test sets, size and how split)

(State what your test results are with the chosen method, parameters: e.g. "We have obtained the best results with the ….. classifier (parameters=....) , giving classification accuracy of …% on test data….")

(Comment on feature importances of models)

(Comment on anything that you deem important/interesting)


You will get full points from here as long as you have a good (enough) summary of your work, regardless of your best performance or what you have decided to talk about in the last few lines.



# **REPORT**

### **PROBLEM DEFINITION**
The aim of this project is to build naive bayes and logistic regression classifier models to **classify** the documents according to their textual content.

### **TEXT CLEANING & PREPROCESSING**

To improve the performace of the model,avoid noise and overfitting it is a good practice to clean or preprocess text data.
We apply the following preprocessing to the data:

1. **Lower case:** The word "Plant", "plant" and "PLANT" are actually the same word amd converting them all to one format i.e lowercase will be more efficient and will prevent the model to overfit data for the same word.
2. **Stopwords:** Stop words like "is", "this" etc are removed because they add noise and have low information that they provide to the model.Stopwords do not add anything to the meaning of a sentence and can removed without causing probelms while training the model. 
3. **SnowballStemmer**: Stemming gets the root form of a word removing for example affixes like -ed, -ize, -de, -s etc added to the end of words. The stem words are created by removing the prefix or suffix of a word.
We need to convert the words firstly into tokens if they are not already tokens and then we covert themt to their root forms.This is also done to prevent the model from overlearning./overfitting the same word that is used in difeerent forms only.

4. **Numbers,Special Characters,Punctuation**: These do not provide any additional information to a text for classigication.

### **TRAIN AND TEST DATA SPLITING**
We import two data sets already that are split into two train and test sets. We sepatate the data and target for both train and test data and apply preprocessing to the data of both train and test. 

### **PERFORMANCE AND RESULTS**

We have obtained the best results with the **Naive Bayes classifier** **(parameters= alpha =0.5)** , giving classification accuracy of **75.5%** on test data.
  
### **FEATURE IMPORTANCE OF MODELS**
Feature importance is carried out to find the most frequently occuring features in a topic/caterogry that assist the model in prediciting then topic/categories/targte_group.

From the documentation, we can use these attributes of the models:

**.feature_log_prob_** or **coef_** to find the probility of a feature(from the vectorizer) observed given a class. For Naiver Bayes we can use either of these of these. 

While from the documnetation of Logistic Regression :
**coef_** attribute finds the Coefficient of the features in the decision function.

In the above both functions `feature_impo` we first used the respective attributes of the models to get an ndarray for all features with their coefficitents.From that list we get the indexes of the 3 most freqently occuring words/features in ascending order wrt to the frequency/probabilty of occurence.To display then in descending ordere we reverse the list.
We add these lists under their respective columns.

### **OBSERVATIONS**

The 3 most important words or features are different for both naive bayes and logistic regression for prediction.




# **Submission**
You will submit this homework via SUCourse.


Please read this document again before submitting it.

Please submit your **"share link" INLINE in Sucourse submissions.** That is we should be able to click on the link and go there and run (and possibly also modify) your code.

For us to be able to modify, in case of errors etc, you should get your "share link" as **share with anyone in edit mode** 

Download the **.ipynb and the .html** file and upload both of them to Sucourse.
 
Please do your assignment individually, do not copy from a friend or the Internet. Plagiarized assignments will receive -100.
